# Import All librarys

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from pandas.plotting import scatter_matrix
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

: 

In [ ]:
# read file
cars = pd.read_csv("D:\Code File\Project\Car_F_and_P\Car F and P.csv")
# for preprocessing and machin learning
train, test = train_test_split(cars, test_size=0.2, random_state=3)
# for general view
cars_view = cars.copy()

cars.shape, train.shape, test.shape

In [ ]:
cars_view.info()

In [ ]:
cars.head()

In [ ]:
cars_view.describe()

In [ ]:
cars_view.hist(bins=50, figsize=(10,7))
plt.show()

In [ ]:
cars_view["Make"].unique()

In [ ]:
# To understand the number of values
cars_view["Engine Fuel Type"].unique()

In [ ]:
# To understand the number of values
cars_view["Vehicle Size"].value_counts()

In [ ]:
# To understand the number of UNKNOWN
cars_view["Transmission Type"].value_counts()

# Object missing fix

# Fix Unknown value in Transmision Type column

In [ ]:
cars_view[cars_view["Transmission Type"] == "UNKNOWN"].head()

In [ ]:
# Display the average of the unknown number over the year for unknown fix
cars_view[cars_view["Year"] > 2000]["Transmission Type"].value_counts()

In [ ]:
# fix unknown value in Transmission Type
transmision_for = cars_view["Transmission Type"].copy()
year = cars_view["Year"].copy()
transmision_type = []               # new values
cars_view = cars_view.drop("Transmission Type", axis=1)

number_ix = 0   # index numbers
for i in transmision_for:
    # because average of the unknown number over the year
    if transmision_for[number_ix] == "UNKNOWN" and year[number_ix] <= 2000:
        transmision_type.append("MANUAL")
        
    elif transmision_for[number_ix] == "UNKNOWN" and year[number_ix] > 2000:
        transmision_type.append("AUTOMATIC")
        
    else:
        transmision_type.append(i)
        
    number_ix += 1

# Merge Columns
cars_view.insert(6, "Transmission Type", transmision_type)
cars_view.head()

In [ ]:
cars_view["Transmission Type"].unique()

# Fix Missing value in Engin Fuel Type

In [ ]:
# view models car
cars_view[cars_view["Engine Fuel Type"].isnull() == True]

In [ ]:
# fix missing value
cars_view["Engine Fuel Type"].fillna("regular unleaded", inplace=True)

cars_view.info()

In [ ]:
cars_view[cars_view["Model"] == "Verona"].head()

# encode object value
(encode object value for better view and better implementation for preprocessing and machin learning)

In [ ]:
# ordinal encoder (Because the label encoder cannot be referenced in the pipeline madule)
encoder = OrdinalEncoder(handle_unknown='error')
feature = ["Make", "Engine Fuel Type", "Transmission Type", "Driven_Wheels", "Vehicle Size", "Vehicle Style"]

encode_tmp = encoder.fit_transform(cars_view[feature])
cars_view_encode = pd.DataFrame(encode_tmp, columns=feature)

cars_view = cars_view.drop(feature, axis=1)
end_columns = cars_view[["highway MPG", "city mpg", "Popularity", "MSRP"]].copy()
cars_view = cars_view.drop(["highway MPG", "city mpg", "Popularity", "MSRP"], axis=1)

cars_view = pd.concat([cars_view, cars_view_encode, end_columns], axis=1)
cars_view.info()

In [ ]:
cars_view.head()

In [ ]:
correlations = cars_view.drop(["Model", "Market Category"], axis=1).corr()
correlations["MSRP"].sort_values(ascending=False)

In [ ]:
# show other corrilation over the MSRP
feature = ["Driven_Wheels", "Popularity", "Number of Doors", "city mpg", "Transmission Type", "Engine HP", "MSRP"]
scatter_matrix(cars_view[feature], figsize=(10,8))
plt.show()

In [ ]:
cars_view.plot(kind='scatter', x="MSRP", y="Engine HP", figsize=(10,7), alpha=0.5)
plt.show()

In [ ]:
cars[cars_view["Engine HP"] > 800]

In [ ]:
cars[cars_view["MSRP"] > 1000000]

In [ ]:
# Show ratio of columns with blank values to the remaining columns
feature = ["Year", "Engine HP", "Engine Cylinders", "Number of Doors", "Popularity", "Engine Fuel Type"]
scatter_matrix(cars_view[feature], figsize=(10,7))
plt.show()

In [ ]:
cars_view.plot(kind='scatter', x="Engine HP", y="Year", figsize=(10,7))
plt.show()

# Preprocessing
1. Removing model and Market Category column because to high number of objects
2. missing value in Engine Fuel Type, Engine HP, Engine Cylinders, Number of Doors, Transmission Type

### Tip
Presence of unknown in Transmission Type

# Pre Processing

In [ ]:
# pre processing 
df = train.copy()

# label column set rows on other column
df_y_upp = df[df["Year"] >= 2010].copy()
df_y_low = df[df["Year"] < 2010].copy()
df_y = pd.concat([df_y_upp, df_y_low], axis=0)
df_y.reset_index(drop=True, inplace=True)
df_y = df_y["MSRP"].copy()

df.drop("MSRP", axis=1, inplace=True)

# Extracting numeric values from a string
df_num = df[["Year", "Engine HP", "Engine Cylinders", "Number of Doors", "highway MPG", "city mpg", "Popularity"]].copy()

df_cat = df[["Year", "Engine Fuel Type", "Transmission Type", "Driven_Wheels", "Vehicle Size", "Vehicle Style"]].copy()

# Engine HP missing value fix over year
imputer = SimpleImputer(missing_values=np.nan, strategy='median')  
 
df_hp_imputed_upp = imputer.fit_transform(df_num[df_num["Year"] >= 2010][["Engine HP"]])
df_hp_imputed_upp = pd.DataFrame(df_hp_imputed_upp)
df_hp_imputed_low = imputer.fit_transform(df_num[df_num["Year"] < 2010][["Engine HP"]])
df_hp_imputed_low = pd.DataFrame(df_hp_imputed_low)
df_hp_imputed = pd.concat([df_hp_imputed_upp, df_hp_imputed_low], axis=0)
df_hp_imputed.reset_index(drop=True, inplace=True)
df_hp_imputed.columns = ["Engine HP"]

# set all rows in Engin HP
df_num_upp = df_num[df_num['Year'] >= 2010].copy()
df_num_low = df_num[df_num['Year'] < 2010].copy()
df_num = pd.concat([df_num_upp, df_num_low], axis=0)
df_num.reset_index(drop=True, inplace=True)

# concatinate Engin Hp to numeric values
df_num.drop("Engine HP", axis=1, inplace=True)
df_num = pd.concat([df_num, df_hp_imputed], axis=1)

# missing fix all numeric
df_imputed = pd.DataFrame(imputer.fit_transform(df_num), columns=df_num.columns)

# category fix
# fix rows to numerical values
df_cat_upp = df_cat[df_cat["Year"] >= 2010].copy()
df_cat_low = df_cat[df_cat["Year"] < 2010].copy()
df_cat = pd.concat([df_cat_upp, df_cat_low], axis=0)
df_cat.reset_index(drop=True, inplace=True)

# new columns value fix
new_transmision = []
new_fuel = []

# fix Transmission Unknown Value
for t, y in zip(df_cat["Transmission Type"], df_cat["Year"]):
    if t == "UNKNOWN":
        if y >= 2000:
            new_transmision.append("AUTOMATIC")
        else:
            new_transmision.append("MANUAL")
    else:
        new_transmision.append(t)
    
# fix missing value in Engin Fuel Type    
for f in df_cat["Engine Fuel Type"]:
    if pd.isnull(f):
        new_fuel.append("regular unleaded")
    else:
        new_fuel.append(f)
        
        
df_cat.drop("Year", axis=1, inplace=True)

# add Transmission column value fix
df_cat.drop("Transmission Type", axis=1, inplace=True)
df_cat.insert(2, "Transmission Type", new_transmision)

# add Engine Fuel Type column value fix
df_cat.drop("Engine Fuel Type", axis=1, inplace=True)
df_cat.insert(1, "Engine Fuel Type", new_fuel)

# encode object value 
encoder = OrdinalEncoder(handle_unknown='error')
df_encode = pd.DataFrame(encoder.fit_transform(df_cat), columns=df_cat.columns)

# concat numeric value and category value
df_prepared = pd.concat([df_imputed, df_encode], axis=1)

# standard scaler
scaler = StandardScaler()
df_prepared = pd.DataFrame(scaler.fit_transform(df_prepared), columns=df_prepared.columns)
df_prepared.head()

# Pre Processing With Pipeline

In [ ]:
# Attribute Selectortegy
class AttributeSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attributes):
        self.attr_name = attributes
        
    def fit(self, df):
        return self
    
    # return values for next process in pipeline
    def transform(self, df):
        return df[self.attr_name].values

# hourse power missing value fix over year
class EnginHpFix(BaseEstimator, TransformerMixin):
    def fit(self, df):
        return self
    
    def transform(self, df):
        year_ix, hp_ix = 0, 1        # Engine HP missing value fix over year
        df = pd.DataFrame(df)
        imputer = SimpleImputer(missing_values=np.nan, strategy='median')  
        
        df_hp_imputed_upp = imputer.fit_transform(df[df[year_ix] >= 2010][[hp_ix]])
        df_hp_imputed_upp = pd.DataFrame(df_hp_imputed_upp)
        df_hp_imputed_low = imputer.fit_transform(df[df[year_ix] < 2010][[hp_ix]])
        df_hp_imputed_low = pd.DataFrame(df_hp_imputed_low)
        df_hp_imputed = pd.concat([df_hp_imputed_upp, df_hp_imputed_low], axis=0)
        df_hp_imputed.reset_index(drop=True, inplace=True)

        # set all rows in Engin HP
        df_num_upp = df[df[year_ix] >= 2010].copy()
        df_num_low = df[df[year_ix] < 2010].copy()
        df_num = pd.concat([df_num_upp, df_num_low], axis=0)
        df_num.reset_index(drop=True, inplace=True)

        # concatinate Engin Hp to numeric values
        df_num.drop(hp_ix, axis=1, inplace=True)
        df_num = pd.concat([df_num, df_hp_imputed], axis=1)
        
        return df_num.values
    
# Category value fix
class CategoryValueFix(BaseEstimator, TransformerMixin):
    def fit(self, df):
        return self
    
    def transform(self, df):
        year_ix, fuel_ix, trans_ix = 0, 1, 2     # index column number
        
        # set rows in numeric values
        df = pd.DataFrame(df)

        # save fix value
        new_transmision = []
        new_fuel = []
        
        # fix Transmission Unknown Value
        for t, y in zip(df[trans_ix], df[year_ix]):
            if t == "UNKNOWN":
                if y >= 2000:
                    new_transmision.append("AUTOMATIC")
                else:
                    new_transmision.append("MANUAL")
            else:
                new_transmision.append(t)
            
        # fix missing value in Engin Fuel Type    
        for f in df[fuel_ix]:
            if pd.isnull(f):
                new_fuel.append("regular unleaded")
            else:
                new_fuel.append(f)
                
        df_upp = df[df[year_ix] >= 2010].copy()
        df_low = df[df[year_ix] < 2010].copy()
        
        df = pd.concat([df_upp, df_low], axis=0)
        df.reset_index(drop=True, inplace=True)
        
        df.drop([year_ix, fuel_ix, trans_ix], axis=1, inplace=True)

        # add Engine Fuel Type column value fix
        df.insert(0, 0, new_fuel)

        # add Transmission column value fix
        df.insert(1, 1, new_transmision)
        
        return df.values

# for set y rows on other columns
def y_set(df):
    df_y_upp = df[df["Year"] >= 2010].copy()
    df_y_low = df[df["Year"] < 2010].copy()

    df_y = pd.concat([df_y_upp, df_y_low], axis=0)
    df_y.reset_index(drop=True, inplace=True)
    return df_y["MSRP"]
        
# create df and y column
df = train.copy()
df_y = y_set(df)
     
attr_num = ["Year", "Engine HP", "Engine Cylinders", "Number of Doors", "highway MPG", "city mpg", "Popularity"]
attr_cat = ["Year", "Engine Fuel Type", "Transmission Type", "Driven_Wheels", "Vehicle Size", "Vehicle Style"]

# number preprocess
num_pipeline = Pipeline([
    ("Attribute-selector", AttributeSelector(attributes=attr_num)),
    ("EnginHp-fix", EnginHpFix()),
    ("Simple-Imputer", SimpleImputer(missing_values=np.nan, strategy='median')),
    ("Standard-scaler", StandardScaler())
])

# category preprocess
cat_pipeline = Pipeline([
    ("Attribute-selector", AttributeSelector(attributes=attr_cat)),
    ("value_fix", CategoryValueFix()),
    ("Ordinal-Encoder", OrdinalEncoder(handle_unknown='error')),
    ("Standard-scaler", StandardScaler())
])

final_pipeline = FeatureUnion(transformer_list=[
    ("number_operation", num_pipeline),
    ("category_operation", cat_pipeline)
])

df_prepared_tmp = final_pipeline.fit_transform(df)
df_prepared = pd.DataFrame(df_prepared_tmp)

df_prepared.head()

# test models

In [ ]:
# for better show result in models
def display_score(score, model):
    print("===================", model, "===================")
    print("Your error = ", score)
    print("mean = ", score.mean())
    print("standard deviation = ", score.std())

In [ ]:
# Dicision Tree
dec_tree = DecisionTreeRegressor()
dec_tree.fit(df_prepared, df_y)

dec_predict = dec_tree.predict(df_prepared)
mse = mean_squared_error(dec_predict, df_y)
rmse = np.sqrt(mse)
rmse

In [ ]:
# Random Forest Regresion
rand_forest = RandomForestRegressor()

score = cross_val_score(rand_forest, df_prepared, df_y, scoring='neg_mean_squared_error', cv=10)
score_rmse = np.sqrt(-score)

display_score(score_rmse, "Random Forest Regressor")

In [ ]:
# Decision Tree On cross validation
dec_reg = DecisionTreeRegressor()

score = cross_val_score(dec_reg, df_prepared, df_y, scoring='neg_mean_squared_error', cv=10)
score_rmse = np.sqrt(-score)

display_score(score_rmse, "Decision Tree Regressor")

In [ ]:
# XGboost Regression model 
xgb = XGBRegressor()

score = cross_val_score(xgb, df_prepared, df_y, scoring='neg_mean_squared_error', cv=10)
rmse_score = np.sqrt(-score)

display_score(rmse_score, "XGB Regressor")

In [ ]:
# search best parametrs for xgboost regressor
xbg = XGBRegressor()

params = {"n_estimators": [200, 300, 400, 500], "max_depth": [3, 5, 7, 9]}
xgb_gd_search = GridSearchCV(xgb, params, scoring='neg_mean_squared_error', cv=10)
xgb_gd_search.fit(df_prepared, df_y)

print("best params", xgb_gd_search.best_params_)
print("Best Estimator = ", xgb_gd_search.best_estimator_)

In [ ]:
# rand_forest = RandomForestRegressor()

# params = {"n_estimators": [100, 200, 300, 400], "max_features": [4, 6, 8, 12]}
# gd_search = GridSearchCV(rand_forest, params, scoring='neg_mean_squared_error', cv=10)
# gd_search.fit(df_prepared, df_y)

# print("best params", gd_search.best_params_)
# print("Best Estimator = ", gd_search.best_estimator_)

# test process 

In [ ]:
model = RandomForestRegressor(n_estimators=200)
model.fit(df_prepared, df_y)

# create test df
test_df = test.copy()

test_y = y_set(test_df)

# preprocessing test df
test_prepared = final_pipeline.transform(test_df)
# predict values
test_predict = model.predict(test_prepared)
# mean score
mse_test = mean_squared_error(test_y, test_predict)
rmse_test = np.sqrt(mse_test)
rmse_test

# final class for py file

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import RandomForestRegressor
from pathlib import Path


# Attribute Selectortegy
class AttributeSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attributes):
        self.attr_name = attributes
        
    def fit(self, df):
        return self
    
    # return values for next process in pipeline
    def transform(self, df):
        return df[self.attr_name].values

# hourse power missing value fix over year
class EnginHpFix(BaseEstimator, TransformerMixin):
    def fit(self, df):
        return self
    
    def transform(self, df):
        year_ix, hp_ix = 0, 1        # Engine HP missing value fix over year
        df = pd.DataFrame(df)
        imputer = SimpleImputer(missing_values=np.nan, strategy='median')  
        
        df_hp_imputed_upp = imputer.fit_transform(df[df[year_ix] >= 2010][[hp_ix]])
        df_hp_imputed_upp = pd.DataFrame(df_hp_imputed_upp)
        df_hp_imputed_low = imputer.fit_transform(df[df[year_ix] < 2010][[hp_ix]])
        df_hp_imputed_low = pd.DataFrame(df_hp_imputed_low)
        df_hp_imputed = pd.concat([df_hp_imputed_upp, df_hp_imputed_low], axis=0)
        df_hp_imputed.reset_index(drop=True, inplace=True)

        # set all rows in Engin HP
        df_num_upp = df[df[year_ix] >= 2010].copy()
        df_num_low = df[df[year_ix] < 2010].copy()
        df_num = pd.concat([df_num_upp, df_num_low], axis=0)
        df_num.reset_index(drop=True, inplace=True)

        # concatinate Engin Hp to numeric values
        df_num.drop(hp_ix, axis=1, inplace=True)
        df_num = pd.concat([df_num, df_hp_imputed], axis=1)
        
        return df_num.values
    
# Category value fix
class CategoryValueFix(BaseEstimator, TransformerMixin):
    def fit(self, df):
        return self
    
    def transform(self, df):
        year_ix, fuel_ix, trans_ix = 0, 1, 2     # index column number
        
        # set rows in numeric values
        df = pd.DataFrame(df)

        # save fix value
        new_transmision = []
        new_fuel = []
        
        # fix Transmission Unknown Value
        for t, y in zip(df[trans_ix], df[year_ix]):
            if t == "UNKNOWN":
                if y >= 2000:
                    new_transmision.append("AUTOMATIC")
                else:
                    new_transmision.append("MANUAL")
            else:
                new_transmision.append(t)
            
        # fix missing value in Engin Fuel Type    
        for f in df[fuel_ix]:
            if pd.isnull(f):
                new_fuel.append("regular unleaded")
            else:
                new_fuel.append(f)
                
        df_upp = df[df[year_ix] >= 2010].copy()
        df_low = df[df[year_ix] < 2010].copy()
        
        df = pd.concat([df_upp, df_low], axis=0)
        df.reset_index(drop=True, inplace=True)
        
        df.drop([year_ix, fuel_ix, trans_ix], axis=1, inplace=True)

        # add Engine Fuel Type column value fix
        df.insert(0, 0, new_fuel)

        # add Transmission column value fix
        df.insert(1, 1, new_transmision)
        
        return df.values

class DataMining(TransformerMixin, BaseEstimator):
    __model = None
    __base_value = None
    __base_cars = None
    
    # for set y rows on other columns
    def y_set(self, df):
        df_y_upp = df[df["Year"] >= 2010].copy()
        df_y_low = df[df["Year"] < 2010].copy()

        df_y = pd.concat([df_y_upp, df_y_low], axis=0)
        df_y.reset_index(drop=True, inplace=True)
        return df_y["MSRP"]
    
    # preproccessing operation
    def preprocessing(self, df):
        attr_num = ["Year", "Engine HP", "Engine Cylinders", "Number of Doors", "highway MPG", "city mpg", "Popularity"]
        attr_cat = ["Year", "Engine Fuel Type", "Transmission Type", "Driven_Wheels", "Vehicle Size", "Vehicle Style"]

        # number preprocess
        num_pipeline = Pipeline([
            ("Attribute-selector", AttributeSelector(attributes=attr_num)),
            ("EnginHp-fix", EnginHpFix()),
            ("Simple-Imputer", SimpleImputer(missing_values=np.nan, strategy='median')),
            ("Standard-scaler", StandardScaler())
        ])

        # category preprocess
        cat_pipeline = Pipeline([
            ("Attribute-selector", AttributeSelector(attributes=attr_cat)),
            ("value_fix", CategoryValueFix()),
            ("Ordinal-Encoder", OrdinalEncoder(handle_unknown='error')),
            ("Standard-scaler", StandardScaler())
        ])

        final_pipeline = FeatureUnion(transformer_list=[
            ("number_operation", num_pipeline),
            ("category_operation", cat_pipeline)
        ])
        
        df_prepared = final_pipeline.fit_transform(df)
        df_prepared = pd.DataFrame(df_prepared)
        
        return df_prepared
    
    def fit(self, df):
        global cars
        cars = pd.read_csv('D:/Code File\Project/Car_F_and_P/Car F and P.csv')
        train, te = train_test_split(cars, train_size=0.8, random_state=2)
        
        # preproccessing firs data and new data 
        cars_prepared = self.preprocessing(train)
        
        # set y first data for learn michin
        cars_y = self.y_set(train)
        
        # learn model
        rand_forest = RandomForestRegressor(n_estimators=200)
        rand_forest.fit(cars_prepared, cars_y)
        
        self.__model = rand_forest     # save model for use in transform
        
        return self
        
    # use new data frame and result
    def transform(self, df):
        # Both for small data and for accurate prediction
        base = cars
        base["find"] = False
        df["find"] = True
        
        df = pd.concat([base, df], axis=0)
        
        # set base value in price predict
        value_upp = df[df["Year"] >= 2010].copy()
        value_low = df[df["Year"] < 2010].copy()

        df_set = pd.concat([value_upp, value_low], axis=0)
        df_set.reset_index(drop=True, inplace=True)
        find_ix = df_set[df_set["find"] == True].index
        
        # save value 
        df_set.drop("find", axis=1, inplace=True)
        self.__base_value = df_set.loc[find_ix]
        self.__base_value.reset_index(drop=True, inplace=True)

        df_x = self.preprocessing(df)
        df_x = df_x.loc[find_ix]
        
        model = self.__model
        
        # predict price
        df_y = model.predict(df_x)
        df_y = pd.DataFrame(df_y, columns=["Price"])
        
        # final data frame
        df_predicted = pd.concat([self.__base_value, df_y], axis=1)
        
        return df_predicted
    
    
testes = pd.read_csv("Car F and P Test.csv")
testes = testes[3000:3001]
dm = DataMining()
t_pred = dm.fit_transform(testes)
t_pred.head()